# Visualizing the Revenue Equation

We want a way to visualize the revenue equation of the revenue management problem. When we take a step in our optimization, we want to see the benefit symbolically.

This notebook serves as a playground to work with the problem and visualize the mathematical nature of the optimization.

In [140]:
# Python stdlib imports
from itertools import combinations

# library imports
import numpy as np
from sympy import *
from sympy.matrices.dense import matrix_multiply_elementwise

# IPython imports (for LaTeX)
from IPython.display import display, Latex

# pulp (Python LP solver, interfaces with CPLEX/Gurobi/other solvers)
from pulp import *

## Symbolic Manipulation Functions

In [141]:
def calculate_symbolic_availability(p, y, N, m):
    m = len(p)
    a = np.triu(np.ones((m, N), dtype=object), 0)

    for i in range(1, m):
        a[i, 0] = (1 - p[i - 1] * y[i - 1]) * a[i - 1, 0]

    # for N_j = 2 and above
    for n in range(1, N):
        for i in range(n + 1, m):
            for tup in combinations(range(i), n):
                prob = 1
                for k in range(i):
                    if k in tup:
                        prob *= p[k] * y[k]
                    else:
                        prob *= (1 - p[k] * y[k])
                a[i, n] += prob

    a = a.sum(axis=1)

    # enforce first N nurses get availability 1
    for i in range(N):
        a[i] = 1
        
    return Matrix(a)

In [142]:
def display_latex(expr):
    display(Latex(f'${latex(expr)}$'))

In [143]:
def show_revenue_function(m, N):
    # r - the revenue of the shift
    r = symbols('r')
    q = Matrix(symbols(f'q1:{m+1}'))
    y = Matrix(symbols(f'Y1:{m+1}'))
    p = Matrix(symbols(f'p1:{m+1}'))

    # now, we calculate availability
    a = calculate_symbolic_availability(p, y, N, m)

    pq = matrix_multiply_elementwise(p, q)
    pqr = pq * r
    pqry = matrix_multiply_elementwise(pqr, y)
    R = sum(matrix_multiply_elementwise(a, pqry))

    # simplify the expression
    return expand(simplify(R)), y

## Experimentation

In [144]:
r, y = show_revenue_function(4, 2)
display_latex(r)


<IPython.core.display.Latex object>